# Notebook Title

## Setup Python and R environment
you can ignore this section

In [3]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [4]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 download your data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [6]:
#Motor vehicle collisions 2023, dropped rows without lat/long as there were 7300 rows and I dont have API credits https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data
import pandas as pd

df = pd.read_csv('df_cleaned.csv')

In [7]:
df =df.sample(10000)

In [8]:
print(df.dtypes)

CRASH DATE                        object
CRASH TIME                        object
BOROUGH                           object
ZIP CODE                         float64
LATITUDE                         float64
LONGITUDE                        float64
LOCATION                          object
ON STREET NAME                    object
CROSS STREET NAME                 object
OFF STREET NAME                   object
NUMBER OF PERSONS INJURED          int64
NUMBER OF PERSONS KILLED           int64
NUMBER OF PEDESTRIANS INJURED      int64
NUMBER OF PEDESTRIANS KILLED       int64
NUMBER OF CYCLIST INJURED          int64
NUMBER OF CYCLIST KILLED           int64
NUMBER OF MOTORIST INJURED         int64
NUMBER OF MOTORIST KILLED          int64
CONTRIBUTING FACTOR VEHICLE 1     object
CONTRIBUTING FACTOR VEHICLE 2     object
CONTRIBUTING FACTOR VEHICLE 3     object
CONTRIBUTING FACTOR VEHICLE 4     object
CONTRIBUTING FACTOR VEHICLE 5     object
COLLISION_ID                       int64
VEHICLE TYPE COD

## 👉 convert addresses --> lat/long 

See the [census-examples](https://github.com/data4news/census-examples) repository for examples. If you need help, try asking in the class slack channel. Chances are someone in the class is struggling with the same problem as you are so we might as well all learn together in the same slack channel! 

In [9]:
# Save the df_cleaned DataFrame as a CSV file
df.to_csv('df_cleaned3.csv', index=False)


## 👉 convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

In [10]:
# Code adapted from:
# https://gis.stackexchange.com/questions/363830/applying-the-censusgeocode-package-to-an-entire-dataframe-of-geocoded-data
# Defines a geocode function that accepts lat/long and spits out geographies
# The code then runs that funciton in parllel (for speed).

import pandas as pd
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import glob
import json
import requests
import pandas as pd
from pprint import pprint
from tqdm import tqdm


import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df['LATITUDE']
        longitudes = df['LONGITUDE']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df)))

    #return pd.DataFrame(data)
        return pd.DataFrame([x for x in data if x])

census_geos_df = bulk_geocode(df['LATITUDE'], df['LONGITUDE']) 
census_geos_df.head()
len(census_geos_df)


  1%|          | 108/10000 [00:01<02:23, 69.05it/s]

Error geocoding (0.0, 0.0): 'Census Blocks'


  3%|▎         | 329/10000 [00:03<01:17, 124.87it/s]

Error geocoding (0.0, 0.0): 'Census Blocks'


  5%|▍         | 485/10000 [00:05<02:06, 74.99it/s] 

...

Error geocoding (0.0, 0.0): 'Census Blocks'


 98%|█████████▊| 9810/10000 [01:45<00:01, 132.08it/s]

Error geocoding (0.0, 0.0): 'Census Blocks'


100%|██████████| 10000/10000 [01:46<00:00, 93.47it/s]


9966

In [11]:
census_geos_df

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,0,360050093022015,+40.8113306,2015,0,36,2015,210701006017333,BK,...,009302,-073.9007669,2,7315,0,-073.9007669,G5040,L,U,005
1,,881,360810500002000,+40.7122123,2000,0,36,2000,210701006105299,BK,...,050000,-073.7663734,2,26390,433,-073.7663734,G5040,L,U,081
2,,1,360810717022010,+40.7327035,2010,0,36,2010,210701006103964,BK,...,071702,-073.8682928,2,1459,0,-073.8682928,G5040,L,U,081
3,,0,360050042003000,+40.8259462,3000,0,36,3000,210701006027280,BK,...,004200,-073.8611502,3,6652,0,-073.8611502,G5040,L,U,005
4,,459,360470523002001,+40.7108088,2001,0,36,2001,210701004655170,BK,...,052300,-073.9589642,2,11698,223,-073.9589642,G5040,L,U,047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9961,,409,360470139004000,+40.6694602,4000,0,36,4000,210701004656402,BK,...,013900,-073.9877625,4,18957,197,-073.9877625,G5040,L,U,047
9962,,0,360470414022000,+40.6047307,2000,0,36,2000,210701004653678,BK,...,041402,-073.9683580,2,705,0,-073.9683580,G5040,L,U,047
9963,,626,360050413006001,+40.8724514,6001,0,36,6001,210701006030361,BK,...,041300,-073.8847640,6,9654,219,-073.8847640,G5040,L,U,005
9964,,0,360811385021018,+40.7522835,1018,0,36,1018,210701006104828,BK,...,138502,-073.7447568,1,40713,0,-073.7447568,G5040,L,U,081


In [14]:
import pandas as pd

# Assuming census_geos_df is your existing dataframe
# Create a new column 'GEOID_Modified' with the GEOID values minus the last four digits
census_geos_df['GEOID_Modified'] = census_geos_df['GEOID'].astype(str).str.slice(0, -4)

# Display the updated dataframe to verify the new column
census_geos_df.head(20)


,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY,GEOID_Modified
0,,0,360050093022015,+40.8113306,2015,0,36,2015,210701006017333,BK,...,-073.9007669,2,7315,0,-073.9007669,G5040,L,U,005,36005009302
1,,881,360810500002000,+40.7122123,2000,0,36,2000,210701006105299,BK,...,-073.7663734,2,26390,433,-073.7663734,G5040,L,U,081,36081050000
2,,1,360810717022010,+40.7327035,2010,0,36,2010,210701006103964,BK,...,-073.8682928,2,1459,0,-073.8682928,G5040,L,U,081,36081071702
3,,0,360050042003000,+40.8259462,3000,0,36,3000,210701006027280,BK,...,-073.8611502,3,6652,0,-073.8611502,G5040,L,U,005,36005004200
4,,459,360470523002001,+40.7108088,2001,0,36,2001,210701004655170,BK,...,-073.9589642,2,11698,223,-073.9589642,G5040,L,U,047,36047052300
5,,1298,360610293002000,+40.8639923,2000,0,36,2000,210701008621009,BK,...,-073.9200958,2,19041,448,-073.9200958,G5040,L,U,061,36061029300
6,,355,360470491003000,+40.7047462,3000,0,36,3000,210701004656726,BK,...,-073.9436509,3,11623,138,-073.9436509,G5040,L,U,047,36047049100
7,,563,360470151003001,+40.6653611,3001,0,36,3001,210701004649785,BK,...,-073.9845162,3,18747,257,-073.9845162,G5040,L,U,047,36047015100
8,,14,360050090001005,+40.8187450,1005,0,36,1005,210701006027699,BK,...,-073.8437344,1,37201,1,-073.8437344,G5040,L,U,005,36005009000
9,,1055,360050211003001,+40.8392558,3001,0,36,3001,210701006029500,BK,...,-073.9250567,3,22073,469,-073.9250567,G5040,L,U,005,36005021100


In [15]:
census_geos_df.rename(columns={'GEOID': 'GEOID_long', 'GEOID_Modified': 'GEOID'}, inplace=True)
census_geos_df.head(20)

,SUFFIX,POP100,GEOID_long,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY,GEOID
0,,0,360050093022015,+40.8113306,2015,0,36,2015,210701006017333,BK,...,-073.9007669,2,7315,0,-073.9007669,G5040,L,U,005,36005009302
1,,881,360810500002000,+40.7122123,2000,0,36,2000,210701006105299,BK,...,-073.7663734,2,26390,433,-073.7663734,G5040,L,U,081,36081050000
2,,1,360810717022010,+40.7327035,2010,0,36,2010,210701006103964,BK,...,-073.8682928,2,1459,0,-073.8682928,G5040,L,U,081,36081071702
3,,0,360050042003000,+40.8259462,3000,0,36,3000,210701006027280,BK,...,-073.8611502,3,6652,0,-073.8611502,G5040,L,U,005,36005004200
4,,459,360470523002001,+40.7108088,2001,0,36,2001,210701004655170,BK,...,-073.9589642,2,11698,223,-073.9589642,G5040,L,U,047,36047052300
5,,1298,360610293002000,+40.8639923,2000,0,36,2000,210701008621009,BK,...,-073.9200958,2,19041,448,-073.9200958,G5040,L,U,061,36061029300
6,,355,360470491003000,+40.7047462,3000,0,36,3000,210701004656726,BK,...,-073.9436509,3,11623,138,-073.9436509,G5040,L,U,047,36047049100
7,,563,360470151003001,+40.6653611,3001,0,36,3001,210701004649785,BK,...,-073.9845162,3,18747,257,-073.9845162,G5040,L,U,047,36047015100
8,,14,360050090001005,+40.8187450,1005,0,36,1005,210701006027699,BK,...,-073.8437344,1,37201,1,-073.8437344,G5040,L,U,005,36005009000
9,,1055,360050211003001,+40.8392558,3001,0,36,3001,210701006029500,BK,...,-073.9250567,3,22073,469,-073.9250567,G5040,L,U,005,36005021100


## 👉 Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).

In [16]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
census_geos_df = census_geos_df[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,36005009302,36,005,009302,2015
1,36081050000,36,081,050000,2000
2,36081071702,36,081,071702,2010
3,36005004200,36,005,004200,3000
4,36047052300,36,047,052300,2001
...,...,...,...,...,...
9961,36047013900,36,047,013900,4000
9962,36047041402,36,047,041402,2000
9963,36005041300,36,005,041300,6001
9964,36081138502,36,081,138502,1018


In [17]:
df_with_geos = pd.concat(
    [ 
        df.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,GEOID,STATE,COUNTY,TRACT,BLOCK
0,2023-10-04,20:39:00,BRONX,10455.0,40.813095,-73.89827,"(40.813095, -73.89827)",BRUCKNER BOULEVARD,LEGGETT AVENUE,NaN,...,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,36005009302,36,005,009302,2015
1,2023-03-23,13:45:00,QUEENS,11423.0,40.712887,-73.76595,"(40.712887, -73.76595)",NaN,NaN,91-35 195 STREET,...,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,36081050000,36,081,050000,2000
2,2023-11-28,14:40:00,QUEENS,11373.0,40.732740,-73.86836,"(40.73274, -73.86836)",LONG ISLAND EXPRESSWAY,QUEENS BOULEVARD,NaN,...,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,36081071702,36,081,071702,2010
3,2023-06-09,08:41:00,NaN,NaN,40.826275,-73.85971,"(40.826275, -73.85971)",WHITE PLAINS ROAD,BRUCKNER BOULEVARD,NaN,...,Station Wagon/Sport Utility Vehicle,Tractor Truck Gasoline,NaN,NaN,NaN,36005004200,36,005,004200,3000
4,2023-11-21,10:14:00,BROOKLYN,11211.0,40.710243,-73.95840,"(40.710243, -73.9584)",HAVEMEYER STREET,BORINQUEN PLACE,NaN,...,Sedan,NaN,NaN,NaN,NaN,36047052300,36,047,052300,2001


In [18]:
# Saving the DataFrame as a CSV file
df_with_geos.to_csv('geocoded_col_10k.csv', index=False)
